# Regridding SPC LSRs

In [2]:
import pandas as pd
import xoak
import xarray as xr
import geopandas as gpd
from shapely.ops import nearest_points

### Loading Data

In [3]:
#load the tornado, hail, and wind dataframes
df_tor = gpd.read_file('1950-2022_torn.csv')
df_hail = pd.read_csv('1955-2022_hail.csv')
df_wind = pd.read_csv('1955-2022_wind.csv')

#convert times to pandas datetimes
df_tor['date'] = pd.to_datetime(df_tor['date'])
df_hail['date'] = pd.to_datetime(df_hail['date'])
df_wind['date'] = pd.to_datetime(df_wind['date'])

#subset the dataframes based on the timerange
df_tor = df_tor[(df_tor['date'] >= '1990-10-01') & (df_tor['date'] <= '2005-09-30')]
df_wind = df_wind[(df_wind['date'] >= '1990-10-01') & (df_wind['date'] <= '2005-09-30')]
df_hail = df_hail[(df_hail['date'] >= '1990-10-01') & (df_hail['date'] <= '2005-09-30')]

/tmp/ipykernel_15589/2709948194.py:4: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  df_wind = pd.read_csv('1955-2022_wind.csv')


In [4]:
#set the geometry to the files
df_tor['geometry'] = gpd.points_from_xy(df_tor.slon, df_tor.slat)
df_hail['geometry'] = gpd.points_from_xy(df_hail.slon, df_hail.slat)
df_wind['geometry'] = gpd.points_from_xy(df_wind.slon, df_wind.slat)

In [5]:
#open dummy xarray dataset
ds = xr.open_dataset('/home/scratch/WRF_BCC/severe_weather/UP_HELI_MAX/historical/1990-1991/UP_HELI_MAX_historical-1990-1991_1990-10-01.nc')
ds = ds.sel(Time ='1990-10-01T00:00:00.000000000')
ds_copy = ds.copy()

In [6]:
#load and coarsen the geog file
geog = xr.open_dataset("/home/scratch/WRF_BCC/geography/geo_em.d01.nc")
geog = geog[['CLAT', 'CLONG']].coarsen(south_north=20, west_east=20, boundary='trim').mean()

#coarsen dummy xarray file and merge the two files togther
ds = ds.coarsen(south_north=20, west_east=20, boundary='trim').sum()
ds = xr.merge([ds, geog.squeeze()])
ds = ds.rename({"CLONG": 'lon', 'CLAT': 'lat'})

#restructure the new dataset
ds = ds.assign_coords({'x': ds.west_east, 'y': ds.south_north})
ds = ds.assign_coords({'lon': ds.lon, 'lat': ds.lat})
ds.xoak.set_index(['lat', 'lon'], 'sklearn_geo_balltree')

In [13]:
#create the WRF dataframe
#flatten the values of the lat/long
df_wrf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(ds.lon.values.flatten(), ds.lat.values.flatten()))
df_wrf['lat'] = ds.lat.values.flatten()
df_wrf['lon'] = ds.lon.values.flatten()

### Nearest Point

In [15]:
#change df variable names
gpd1 = df_tor
gpd2 = df_wrf

In [19]:
#defintion to find the nearest point to a point 
pts3 = gpd2.geometry.unary_union

def near(point, pts=pts3):
    
    # find the nearest point and return the corresponding Place value
    nearest = gpd2.geometry == nearest_points(point, pts)[1]
    
    return gpd2[nearest].lat, gpd2[nearest].lon

gpd1['Nearest'] = gpd1.apply(lambda row: near(row.geometry), axis=1)

In [23]:
#save off file
gpd1.to_csv(f'/home/scratch/jcorner1/syn_sev/dataframes/SPC_LSRs_regridded_tor.csv')